In [0]:
DECLARE OR REPLACE VARIABLE schema_use STRING DEFAULT 'nebraska_medicine_demo'

In [0]:
USE IDENTIFIER("mcutini." || schema_use);

Depending on the source of the FHIR bundle it may include its own metadata.  This is standard practice for providers that clean and standardize your incoming FHIR data, or convert your C-CDAs, ADTs, or other clinical data to the FHIR format such as Redox.  

Synthea doesn't include bundle metadata, therefore we're not going to fully parse the Meta object quite as much as we would if it were available.  We do have one Redox JSON loaded into our bronze tables that we can at least review.  Normally we'd split the Meta object object into separate columns exactly like we would for the Bundle's "entry resources" but that would be overkill for this class. 

What is helpful however is to have a place for items such as the `file_metadata` and then `ingest_time` avaialble in a silver table so that we don't need to continue to bring that along for the ride with the Resource tables.  

In [0]:
%skip
DROP TABLE IF EXISTS bundle_metadata;

In [0]:
%skip
CREATE OR REFRESH STREAMING TABLE bundle_metadata (
  bundle_uuid STRING PRIMARY KEY COMMENT 'Unique identifier for the FHIR bundle and the primary key for the table.  This column will be required for all joins amongst resource tables as Resource primary keys are only guaranteed to be unique inside of a bundle.'
, file_metadata STRUCT<
    file_path: STRING
  , file_name: STRING
  , file_size: BIGINT
  , file_block_start: BIGINT
  , file_block_length: BIGINT
  , file_modification_time: TIMESTAMP> COMMENT 'Original metadata of the file ingested from the volume.'
, ingest_time TIMESTAMP
, bundle_resourceType STRING COMMENT "The FHIR Bundle's overall resource type."
, bundle_type STRING COMMENT "The FHIR Bundle's overall type."
, meta VARIANT COMMENT 'Metadata about the FHIR Bundle overall, typically includes information about the source of the bundle, but is not required to be passed for a valid HL7 transaction.  Always NULL for data that originated from Synthea.'
)
COMMENT 'Original FHIR Bundle Metadata'
TBLPROPERTIES (
  'delta.enableChangeDataFeed' = 'true'
  ,'delta.enableDeletionVectors' = 'true'
  ,'delta.enableRowTracking' = 'true'
  ,'quality' = 'bronze'
  ,'pipelines.channel' = 'PREVIEW'
  ,'delta.feature.variantType-preview' = 'supported'
)
AS SELECT
  bundle_uuid
, file_metadata
, ingest_time
, fhir:resourceType::string as bundle_resourceType
, fhir:type::string as bundle_type
, fhir:Meta as meta
FROM STREAM (fhir_bronze_variant)
WHERE file_metadata.file_name = 'DBPriorAuthExample.json'

## Extracting and Exploding the FHIR Resources 
***

In the FHIR bundle, Entry is an array of the bundle's resources, which is the primary data that we need to extract from the bundle in order to make silver tables.  This will allow us to much more easily integrate the data with other systems, check with duplicates, or apply CDC on any final reporting tables we might create.  

Note that the tables we're going to create here for the Resources are just the beginning.  We will have stream ingested the FHIR bundles to make them easy to query and join together to form the relevant information they were intended to provide us, but these tables are not the final destination of this data.  

For example, as a Health Plan we very likely have different member ids than what the ACO, Medical Home, lab, Hosptial etc uses.  We may even have internal "belly button ids" that represent the member over the course of their lifetime that differs from what's on their Subscriber ID cards.  This sort of specific enterprise enrichment is not going to be part of the FHIR bundles themselves and would occur as further ETL in your Lakehouse.  

These Resource tables simply make that next step much easier, while also giving us a platform for real-time reporting, machine learning and GenAI to take place while the data is in motion.  

The very first thing we'll do is explode the Entry array long, and begin to capture the relevant information that will allow us to filter and then pivot later.  

In [0]:
DROP TABLE IF EXISTS fhir_bronze_resources;

We're able to explode our variant data arrays long by using the `variant_explode` function.  Note that in SQL, we're exploding as part of the `FROM` statement.  This allows us to write SQL against the exploded data as though it were the source of another VARIANT table.  The `LATERAL` statement automatically takes care of any grouping or joining that would otherwise be required.  

In [0]:
CREATE MATERIALIZED VIEW fhir_bronze_resources (
  bundle_uuid STRING NOT NULL COMMENT 'Unique identifier for the FHIR bundle.'
, resource_uuid STRING NOT NULL PRIMARY KEY COMMENT 'Unique identifier for the FHIR Resource in a bundle.'
, fullUrl STRING NOT NULL COMMENT 'The full URL of the resource in the entry array.  This serves as the primary key for the resource and later is used to join related resources from within a bundle.'
, resourceType STRING NOT NULL COMMENT 'The type of resource from the bundles entry array.'
, pos INT COMMENT 'The position of the resource element from within the specific resource itself.  Note that a bundle my have many of the same resources inside the entry array and positioning may not be consistent across resources or bundles.'
, key STRING NOT NULL COMMENT 'The name of the resources element extracted. These will serve as column names in the resource target tables.'
, value VARIANT COMMENT 'The value of the resource element extracted.  This will be a variant type and may contain nested variants.'
-- , CONSTRAINT fk_bundle_meta_bundle_uuid FOREIGN KEY (bundle_uuid) REFERENCES nemcutini.nebraska_medicine_demo.bundle_metadata(bundle_uuid) -- Skipped creating this table
)
COMMENT 'Exploded FHIR Resources'
TBLPROPERTIES (
  'delta.enableChangeDataFeed' = 'true'
  ,'delta.enableDeletionVectors' = 'true'
  ,'delta.enableRowTracking' = 'true'
  ,'quality' = 'bronze'
  ,'pipelines.channel' = 'PREVIEW'
  ,'delta.feature.variantType-preview' = 'supported'
)
AS SELECT
  sha2(concat(bundle_uuid, entry.value:fullUrl::string), 256) as resource_uuid
, bundle_uuid
, CAST(entry.value:fullUrl AS STRING) as fullUrl
, CAST(entry.value:resource.resourceType AS STRING) as resourceType
, resource.*
FROM 
  fhir_bronze_variant
, LATERAL variant_explode(fhir:entry) as entry
, LATERAL variant_explode(entry.value:resource) as resource


In [0]:
%skip
SELECT * FROM fhir_bronze_resources;

Variant makes our job of parsing even the most complex JSON files much easier, however at some point we want to be able to apply data types to the data that we've extracted instead of having to do it directly with a cast each time we want to select an element.  

This is especially true when we want to be able to aggregate different pieces of the data together for AI/BI dashboards, or use the output for machine learning models.  

There are other functions that make more sense to use now that fhir_bronze_resource's 'value' column (of type VARIANT) contains much smaller values. 

* `schema_of_variant` returns a string value of the schema for a variant object. 
* `schema_of_variant_agg` may be used to get a more general schema of the variant object by aggregating it over a grouping variable, such as the *resourceType* and *key* from our `fhir_resources` table.  

In [0]:
DROP TABLE IF EXISTS fhir_bronze_resource_schemas;

In [0]:
CREATE OR REPLACE MATERIALIZED VIEW fhir_bronze_resource_schemas (
  resourceType STRING COMMENT 'The type of resource from the bundles entry array.'
, column_name STRING COMMENT 'The name of the resources element extracted. These will serve as column names in the resource target tables.'
, schema_of_variant STRING COMMENT 'The schema of the resource element extracted as VARIANT.  The schema may be applied to the data elements to change into the elements official datatype.  Note that this is based on what was recieved in the bundles, which may include many nested extentions and is therefore more robust than schemas from the FHIR specefication documents.'
, schema_as_struct STRING COMMENT 'The same schema as the schema_of_variant, but with STRUCT instead of OBJECT. (Note that Matt is still determining if this is needed or not.)'
)
COMMENT 'Schemas of inferred from FHIR Resource Variant Data Types'
TBLPROPERTIES (
  'delta.enableChangeDataFeed' = 'true'
  ,'delta.enableDeletionVectors' = 'true'
  ,'delta.enableRowTracking' = 'true'
  ,'quality' = 'bronze'
  ,'pipelines.channel' = 'PREVIEW'
  ,'delta.feature.variantType-preview' = 'supported'
)
AS SELECT
  resourceType
, key as column_name
, schema_of_variant_agg(value) as schema_of_variant
, REPLACE(schema_of_variant_agg(value), 'OBJECT', 'STRUCT') as schema_as_struct
FROM fhir_bronze_resources
GROUP BY 1, 2